In [23]:
import os
import re
import pandas as pd
import numpy as np

def extract_weka_metrics(base_dir="results/WEKA Outputs/Experiment 3 Results"):
    """
    Recursively searches for WEKA output files in the specified directory,
    extracts class-wise classification metrics including 'Pass', 'No submission', 
    'Fail', and 'Weighted Avg.', and returns a DataFrame.

    Returns:
        pd.DataFrame: DataFrame with columns:
                      ['Challenge', 'Module', 'Method', 'Class', 'TP Rate', 
                       'FP Rate', 'Precision', 'Recall', 'F-Measure']
    """
    section_start = re.compile(r"=== Detailed Accuracy By Class ===")
    metric_line = re.compile(
        r"^\s*([\d?.]+)\s+([\d?.]+)\s+([\d?.]+)\s+([\d?.]+)\s+([\d?.]+)"
    )
    class_names = ['Pass', 'No submission', 'Fail', 'Weighted Avg.']

    data = []

    for root, _, files in os.walk(base_dir):
        for fname in files:
            if fname.endswith("attributeselected-bfs") or fname.endswith("attributeselected-gainratio"):
                path = os.path.join(root, fname)

                parts = root.split(os.sep)
                if len(parts) < 2:
                    continue

                module_name = parts[-1]
                challenge_name = parts[-2]
                method = fname.split("-")[-1]  # 'bfs' or 'gainratio'

                with open(path, 'r', encoding='utf-8') as f:
                    lines = f.readlines()

                for i, line in enumerate(lines):
                    if section_start.match(line.strip()):
                        current_class_idx = 0
                        for j in range(i + 2, min(i + 10, len(lines))):
                            line = lines[j].strip()
                            match = metric_line.match(line)
                            if match and current_class_idx < len(class_names):
                                values = [
                                    float(v) if v != '?' else np.nan
                                    for v in match.groups()
                                ]
                                data.append({
                                    "Challenge": challenge_name,
                                    "Module": module_name,
                                    "Method": method,
                                    "Class": class_names[current_class_idx],
                                    "TP Rate": values[0],
                                    "FP Rate": values[1],
                                    "Precision": values[2],
                                    "Recall": values[3],
                                    "F-Measure": values[4]
                                })
                                current_class_idx += 1
                        break

    return pd.DataFrame(data).sort_values(by=["Challenge", "Module", "Method", "Class"]).reset_index(drop=True)


In [25]:
df = extract_weka_metrics()
df.to_csv('temp.csv', index=False)

In [1]:
def summarize_metrics_by_challenge_and_class(df):
    """
    Computes the average metrics for each (Challenge, Class) combination,
    rounding results to 2 decimal places and ordering by Challenge and Class.

    Args:
        df (pd.DataFrame): Output from `extract_weka_metrics()`.

    Returns:
        pd.DataFrame: Summary DataFrame with averages,
                      rounded and sorted by Challenge and Class.
    """
    summary = (
        df.groupby(["Challenge", "Class"])[
            ["TP Rate", "FP Rate", "Precision", "Recall", "F-Measure"]
        ]
        .mean(numeric_only=True)
        .round(2)
        .reset_index()
        .sort_values(by=["Challenge", "Class"], ascending=[True, False])
        .reset_index(drop=True)
    )
    return summary


In [34]:
summary = summarize_metrics_by_challenge_and_class(df)
summary

,Challenge,Class,TP Rate,FP Rate,Precision,Recall,F-Measure
0,challenge-beginners-2018,Pass,0.96,0.24,0.88,0.96,0.92
1,challenge-beginners-2018,No submission,0.79,0.05,0.91,0.79,0.84
2,challenge-beginners-2018,Fail,0.04,0.00,0.54,0.04,0.11
3,challenge-beginners-blockly-2018,Pass,0.92,0.18,0.90,0.92,0.90
4,challenge-beginners-blockly-2018,No submission,0.85,0.10,0.83,0.85,0.83
5,challenge-beginners-blockly-2018,Fail,0.01,0.00,0.13,0.01,0.05
6,challenge-intermediate-2018,Pass,0.92,0.26,0.88,0.92,0.90
7,challenge-intermediate-2018,No submission,0.76,0.09,0.88,0.76,0.81
8,challenge-intermediate-2018,Fail,0.00,0.00,0.07,0.00,0.01
9,challenge-newbies-2018,Pass,0.79,0.09,0.89,0.79,0.83
